In [1]:
#NN Surrogate model class
import injector_surrogate_quads
from injector_surrogate_quads import *
import physics_gp

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#Pytorch 
import numpy as np
import torch
import transformer
import gpytorch
import botorch 

# BO for Minimizing Emittance with 9 Variables (SQ, CQ, SOL, matching quads)

In [2]:
#load injector model
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

2022-06-10 11:39:37.891109: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Objective Function

In [3]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','CQ01:b1_gradient', 'SQ01:b1_gradient',
                 "QA01:b1_gradient", "QA02:b1_gradient", 
                 "QE01:b1_gradient", "QE02:b1_gradient", "QE03:b1_gradient", "QE04:b1_gradient"]

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(varx, vary, varz, var1, var2, var3, var4, var5, var6): 
    """
    vars: sol, cq, sq, qa1, qa2, q1, q2, q3, q4
    """ 
    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, SQ, CQ to values from optimization step
    x_in[:, Model.loc_in[opt_var_names[0]]] = varx
    x_in[:, Model.loc_in[opt_var_names[1]]] = vary
    x_in[:, Model.loc_in[opt_var_names[2]]] = varz
    x_in[:, Model.loc_in[opt_var_names[3]]] = var1
    x_in[:, Model.loc_in[opt_var_names[4]]] = var2
    x_in[:, Model.loc_in[opt_var_names[5]]] = var3
    x_in[:, Model.loc_in[opt_var_names[6]]] = var4
    x_in[:, Model.loc_in[opt_var_names[7]]] = var5
    x_in[:, Model.loc_in[opt_var_names[8]]] = var6

    #output predictions
    y_out = Model.pred_machine_units(x_in) 

    return -1*objective(y_out)[0]


def objective(y_out):
    
    #output is geometric emittance in transverse plane
    out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
       
    return np.sqrt(out1*out2)/1e-6 # in um units

## Gaussian Regression & Acquisition Function

In [4]:
def get_BO_point(x, f, bounds, beta=2.5, precision = None, phys=True):
    """

    function that trains a GP model of data and returns the next observation point using UCB
    D is input space dimensionality
    N is number of samples

    :param x: input points data, torch.tensor, shape (N,D)
    :param f: output point data, torch.tensor, shape (N,1)
    :param bounds: input space bounds, torch.tensor, shape (2,D)
    :param precision: precision matrix used for RBF kernel (must be PSD), torch.tensor, (D,D)
    :param beta: UCB optimization parameter, float
    :return x_candidate, model: next observation point and gp model w/observations
    """
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    
    if phys == True:
        gp = physics_gp.PhysicsExactGPModel(x, f.flatten(), likelihood, precision)
    else:
        gp = botorch.models.SingleTaskGP(x, f, likelihood)
        
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
    
    # fit GP hyperparameters
    #print('training hyperparameters')
    botorch.fit.fit_gpytorch_model(mll)

    # do UCB acquisition
    #print('optimizing acquisition function')
    UCB = botorch.acquisition.UpperConfidenceBound(gp, beta=beta)
    candidate, acq_value = botorch.optim.optimize_acqf(UCB,
                                                       bounds=bounds,
                                                       q=1,
                                                       num_restarts=20,#5
                                                       raw_samples=20)
    return candidate, gp

## Set up initial training samples

In [15]:
#specify bounds
bounds = {'varx': (0.46, 0.485),
          'vary': (-0.02, 0.02),
          'varz': (-0.02, 0.02),
          'var1': (-4, -1),
          'var2': (1, 4),
          'var3': (-7, -1),
          'var4': (-1, 7),
          'var5': (-1, 7),
          'var6': (-7, 1)}
                        
tbounds = torch.tensor([[bounds['varx'][0], bounds['varx'][1]],
                        [bounds['vary'][0], bounds['vary'][1]], 
                        [bounds['varz'][0], bounds['varz'][1]],
                        [bounds['var1'][0], bounds['var1'][1]],
                        [bounds['var2'][0], bounds['var2'][1]],
                        [bounds['var3'][0], bounds['var3'][1]],
                        [bounds['var4'][0], bounds['var4'][1]],
                        [bounds['var5'][0], bounds['var5'][1]],
                        [bounds['var6'][0], bounds['var6'][1]]])

#print(bounds['varx'][0])
#print(tbounds)

#create initial samples within specified bounds
n_samples = 3
n_var = 9
train_x = torch.zeros((n_samples, n_var)) 
for i in range(n_var):
    train_x[:,i] = torch.as_tensor(np.random.uniform(tbounds[i,0],tbounds[i,1],(n_samples,)))
#train_x = torch.as_tensor(train_x).type(torch.FloatTensor)

"""
train_x = torch.rand(n_samples,3) 
train_x[:,0] = train_x[:,0] * diff['varx'] + bounds['varx'][0]
train_x[:,1] = train_x[:,1] * diff['vary'] + bounds['vary'][0]
train_x[:,2] = train_x[:,2] * diff['varz'] + bounds['varz'][0]
""" 
print(train_x)

train_y = torch.tensor([evaluate(vars[0], vars[1], vars[2], vars[3], vars[4], vars[5], vars[6], vars[7], vars[8]) for vars in train_x]).reshape(-1,1)
print(train_y)


#transformer 
transformer_x = transformer.Transformer(tbounds.transpose(0,1), transform_type = 'normalize')
print(transformer_x.mins)
print(transformer_x.maxs)
normed_train_x = transformer_x.forward(train_x)

transformer_y = transformer.Transformer(train_y, transform_type = 'standardize') 
normed_train_y = transformer_y.forward(train_y)

""" 
#create initial model
gp = SingleTaskGP(normed_train_x, normed_train_y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll);
"""

tensor([[ 4.6451e-01,  1.1897e-02, -9.3423e-03, -2.5061e+00,  2.6963e+00,
         -6.0933e+00, -4.3141e-01,  4.5695e+00, -4.0380e+00],
        [ 4.8401e-01,  1.3666e-02, -1.6710e-03, -2.8301e+00,  3.1892e+00,
         -1.1899e+00,  4.3206e+00, -7.1263e-01, -5.5563e+00],
        [ 4.8401e-01, -4.4038e-03, -1.5552e-02, -2.0217e+00,  3.1575e+00,
         -4.7089e+00,  6.1907e+00,  6.6293e+00,  6.2633e-01]])
tensor([[-1.0803],
        [-0.9628],
        [-1.1367]])
[ 0.46 -0.02 -0.02 -4.    1.   -7.   -1.   -1.   -7.  ]
[ 0.485  0.02   0.02  -1.     4.    -1.     7.     7.     1.   ]


' \n#create initial model\ngp = SingleTaskGP(normed_train_x, normed_train_y)\nmll = ExactMarginalLogLikelihood(gp.likelihood, gp)\nfit_gpytorch_model(mll);\n'

## Bayesian Optimization

In [14]:
n_steps = 30
best_y = torch.max(train_y)
for i in range(n_steps):
    if i > 10:
        phys_l = True
        
    bounds = torch.cat((torch.zeros(1,n_var), torch.ones(1,n_var)), 0)
    #x_new, model = get_BO_point(train_x, normed_train_y, bounds=tbounds.transpose(0,1), phys = False)
    x_new, model = get_BO_point(normed_train_x, normed_train_y, bounds=bounds, phys = False)
    
    #train_x = torch.cat((train_x, x_new))
    train_x = torch.cat((train_x, transformer_x.backward(x_new)))
    normed_train_x = transformer_x.forward(train_x)
    
    new_y = torch.tensor(evaluate(train_x[-1,0], train_x[-1,1], train_x[-1,2], 
                                 train_x[-1,3], train_x[-1,4], train_x[-1,5],
                                 train_x[-1,6], train_x[-1,7], train_x[-1,8])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    normed_train_y = transformer_y.forward(train_y)
    
    train_x = train_x.type(torch.FloatTensor)
    train_y = train_y.type(torch.FloatTensor)
    
    if (new_y > best_y):
        best_y = new_y
        color = '\033[95m', '\033[0m'
    else: 
        color = '\u001b[30m', '\033[0m'
    
    print("iteration     target     varx      vary     varz")
    print(f'{color[0]}{i+1}           {train_y[-1,0]:.5f}   {train_x[-1,0]:.5f}   {train_x[-1,1]:.5f}   {train_x[-1,2]:.5f}   {train_x[-1,3]:.5f}   {train_x[-1,4]:.5f}   {train_x[-1,5]:.5f}   {train_x[-1,6]:.5f}   {train_x[-1,7]:.5f}   {train_x[-1,8]:.5f}{color[1]}')
    #print(f'max = {torch.max(train_y):.5f}')

iteration     target     varx      vary     varz
1           -0.67491   0.47803   -0.00473   0.00971   -3.65972   1.61612   -2.94289   0.94269   3.92756   0.51773
iteration     target     varx      vary     varz
2           -0.89345   0.48093   -0.00406   0.00703   -4.00000   1.28971   -2.33415   -0.13175   3.88196   1.00000
iteration     target     varx      vary     varz
3           -0.63800   0.47443   -0.00247   0.00943   -3.41178   1.85072   -3.53525   1.68525   3.56334   0.24611
iteration     target     varx      vary     varz
4           -0.61523   0.47630   -0.00350   0.00883   -3.33063   1.68117   -3.20960   1.19199   3.73631   -1.22965
iteration     target     varx      vary     varz
5           -0.65180   0.47408   -0.00685   0.00710   -3.06722   1.64585   -3.41565   0.65518   4.17484   0.00839
iteration     target     varx      vary     varz
6           -0.60772   0.47758   -0.00660   0.00568   -3.46981   1.91220   -4.00345   1.13020   3.70196   -0.73856
iteration     targe

In [ ]:
"""
#define acquisition function
from botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement
from botorch.optim import optimize_acqf

#optimize
n_steps = 45
for i in range(n_steps):
    #best_normed_y = torch.max(normed_train_y)
    UCB = UpperConfidenceBound(gp, beta=2.5)
    #EI = ExpectedImprovement(gp, best_normed_y)

    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)
    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)

    train_x = torch.cat((train_x, transformer_x.backward(candidate)))
    normed_train_x = transformer_x.forward(train_x)

    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    
    print("iteration        target         varx         vary         varz")
    print(f'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-1][2]:.5f}')
    print(torch.max(train_y))
    
    transformer_y = transformer.Transformer(train_y, 'standardize')
    normed_train_y = transformer_y.forward(train_y)

    gp = SingleTaskGP(normed_train_x, normed_train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll);
"""